In [1]:
# !pip install joblib
# !pip install gensim
# !pip install glove-python-binary
# !pip install transformers
# !pip install datasets
# !pip install tokenizers

In [2]:
import time
import os
import psutil

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [3]:
import numpy as np
import gc
import pandas as pd
import re
from joblib import Parallel, delayed
from gensim.models import FastText,Word2Vec
from glove import Glove,Corpus
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,PreTrainedTokenizerFast,pipeline)
from datasets import Dataset
from sklearn.linear_model import Ridge
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

In [4]:
from NLP_feature import NLP_feature

In [5]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [6]:
# df_test.head(3)

In [8]:
use_Toknizer=True
emb_mode = 'Word2Vec'
encode_mode = 'supervise'
text_columns_name = ['excerpt']
target_column = df_train['target']
candidate_labels=None

In [9]:
nlp = NLP_feature()
# nlp.tokenizers
nlp.do_mlm = True
nlp.emb_size=100
nlp.n_clusters=20
@count_time
@count_info
def fit():
    return nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)
df = fit()

Fitting column: excerpt tokenizer



Tokenizing column: excerpt
Fitting column: excerpt word2vec embedding
Fitting column: excerpt encoder
程序占用了内存133952.0KB
程序运行了16.439002752304077秒


In [10]:
# for column in df.columns:
#     df_train[column] = df[column]

In [11]:
@count_time
@count_info
def trans():
    return nlp.transform(df_test)
test = trans()

Transforming column: excerpt
Tokenizing column: excerpt
程序占用了内存3804.0KB
程序运行了3.1925668716430664秒


In [12]:
# df_train.head(1)

In [13]:
# df_train.to_csv(f'mlm_{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
# test.to_csv(f'mlm_{emb_mode}_{encode_mode}_autox_tst.csv',index=False)